# Agents for Amazon Bedrock - Associate Knowledge Base to Agent

This notebook provides sample code for associating a Knowledge Base for Amazon Bedrock to an existent Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
Now that we have a data soruce and RAG setup, we also need some useful actions or tools that can perform basic booking in this app. For this we will create a Hotel Booking assistant that allows customers to create, delete or get reservation information. The architecture looks as following:

![Agent Architecture](images/MM-Agents.gif)

### Notebook Walk-through

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Use the already created Knowledge Base and its pre-requirements (including OpenSearch Servelless Collection and Indexes)
- Update Agent IAM role to allow for Knowledge Base access
- Associate Knowledge Base with Hotel Booking Agent
- Test Agent invocation with Knowledge Base access


### Next Steps: 
In the next lab, we will test the agent invocation with Action Group and Knowledge Base requests as well as provide extra information to the agent using Prompt attributes

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [1]:
%store -r

Let's now import the necessary libraries and initiate the required boto3 clients

In [2]:
from knowledge_base import KnowledgeBasesForAmazonBedrock
from agent import invoke_agent_helper
import boto3
import os
import time
import json

In [3]:
region_name = 'us-west-2'
iam_client = boto3.client('iam', region_name=region_name)

session = boto3.session.Session(region_name=region_name)

bedrock_agent_client = session.client('bedrock-agent', 
                                            region_name=region_name)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', 
                                            region_name=region_name)

s3_client = boto3.client('s3', region_name='us-east-1')

In [5]:
kb_name = "knowledge-base-quick-start-ruiliang"
kb_id = "0P698YVCTM"
ds_id = "VLWEBP7VAT"
kb_iam_role_name = "AmazonBedrockExecutionRoleForKnowledgeBase_v24y2"

### Updating Agent role to allow Knowledge Base Retrieve and Retrieve and Generate queries

Now that our Knowledge Base is working, we will associate it with the agent. To do so, we first need to update the agent role to allow for retrieval from context from our knowledge base

In [6]:
kb_policies_statements = [
    {
        "Sid": "QueryKB",
        "Effect": "Allow",
        "Action": [
            "bedrock:Retrieve",
            "bedrock:RetrieveAndGenerate"
        ],
        "Resource": [
            f"arn:aws:bedrock:{region_name}:{account_id}:knowledge-base/{kb_id}"
        ]
    }
]
bedrock_agent_kb_policy_statement = {
    "Version": "2012-10-17",
    "Statement": kb_policies_statements
}
bedrock_agent_kb_policy_json = json.dumps(bedrock_agent_kb_policy_statement)
kb_policy_name = f"{agent_name}-kb-{kb_id}"
agent_kb_policy = iam_client.create_policy(
    PolicyName=kb_policy_name,
    PolicyDocument=bedrock_agent_kb_policy_json
)


In [7]:
iam_client.attach_role_policy(
    RoleName=agent_role['Role']['RoleName'],
    PolicyArn=agent_kb_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '5e5aaa23-5739-4a89-a60c-b4c251943597',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Feb 2025 11:37:05 GMT',
   'x-amzn-requestid': '5e5aaa23-5739-4a89-a60c-b4c251943597',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Associate Knowledge Base with Agent

Finally, we can associate the new knowledge base with the agent using the [`AssociateAgentKnowledgeBase`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/associate_agent_knowledge_base.html) API from boto3

In [8]:
time.sleep(10)
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about the information about the hotel',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

#### Preparing Agent

after updating our agent, we need to prepare it again to package all its new components

In [9]:
alias_id = 'TSTALIASID'

In [10]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
intermediate_agent_status = ['CREATING', 'PREPARING', 'UPDATING', 'VERSIONING']
while bedrock_agent_client.get_agent(agentId=agent_id)['agent']['agentStatus'] in intermediate_agent_status:
    time.sleep(10)


{'ResponseMetadata': {'RequestId': '07142e21-0dbf-4625-b79f-b2576484582b', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 07 Feb 2025 11:37:35 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '07142e21-0dbf-4625-b79f-b2576484582b', 'x-amz-apigw-id': 'FnKD-GDhvHcEK0A=', 'x-amzn-trace-id': 'Root=1-67a5f07f-3757211d72496ec5371ce6d8'}, 'RetryAttempts': 0}, 'agentId': 'WOTFRWMOH8', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 2, 7, 11, 37, 35, 801098, tzinfo=tzlocal())}


### Invoking Agent

Now that our Agent has been updated, let's test it again. To do so we will again use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

We will use the same support function called `invoke_agent_helper` to allow us to invoke the agent with or without trace enabled and with or without session state. We will get into more details about those concepts in the `03_invoke_agent.ipynb` notebook.

Now we can test it by asking a question where the answer is available in the knowledge base documents

In [11]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """I want to book a room in ABC grand hotel, but I know my booking means i am investing in it. 
So lets first find out how ABC Grand has been doing in terms of FFO per dilluted shares?"""
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Here is the financial performance of ABC Grand Hotel for 2023:


Underlying profit increased to US$81 million, from US$8 million in 2022, with underlying earnings per share at US¢6.41, compared with US¢0.60 in 2022. Non-trading losses of US$446 million primarily comprised a non-cash revaluation of the Causeway Bay site under development, resulting in a loss attributable to shareholders of US$365 million. Net debt fell to US$225 million at the end of 2023, from US$376 million at the end of 2022, reflecting significantly higher operating cashflow from the business, net of ongoing capital investment, as well as proceeds from disposals. Gearing as a percentage of adjusted shareholders' funds was 5%, compared to 8% at the end of 2022. The Directors recommend a final dividend of US¢3.50 per share.





Combined total revenue of hotels under management increased by 21% to US$1,890.2 million in 2023, from US$1,568.1 million in 2022. Revenue increased by 23% to US$558.1 million in 2023, from US

### Next Steps

Next we will test our agent with knowledge base

In [12]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Whats exact Adjusted EBITDAre in 2023 and 2022 for ABC Grand in millions?"""
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



The exact Adjusted EBITDA for ABC Grand in 2023 was US$177.6 million and in 2022 was US$111.4 million.


 
CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 5.46 s
